# FILTERING, CLEANING ON COLAB

In [1]:
import csv
import json
import pandas as pd
import numpy as np

import os,shutil, sys
import time
import datetime
from datequarter import DateQuarter

import re
import string
punc = string.punctuation
from ast import literal_eval

In [2]:
from joblib import Parallel, delayed
from tqdm import tqdm
import joblib

In [3]:
import dask
import dask.dataframe as dd

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]")\
     .config("spark.executor.memory", "50g")\
     .config("spark.driver.memory", "25g")\
     .config("spark.memory.offHeap.enabled",True)\
     .config("spark.memory.offHeap.size","12g").appName('sachin').getOrCreate()

In [5]:
from functools import reduce 
from pyspark.sql import DataFrame
from pyspark.sql.functions import sum, col, desc, countDistinct, isnan, when, count, isnull, coalesce, substring, first
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [6]:
os.chdir(r"/content/drive/Shareddrives/DATA 228 Big Data/Project/data/raw_source/NOAA_Weather")
os.listdir()

['2005.csv',
 '2006.csv',
 '2007.csv',
 '2008.csv',
 '2009.csv',
 '2010.csv',
 '2011.csv',
 '2012.csv',
 '2013.csv',
 '2014.csv',
 '2015.csv',
 '2016.csv',
 '2017.csv',
 '2018.csv',
 '2019.csv',
 '2020.csv',
 '2021.csv',
 '2022.csv',
 '2023.csv']

In [8]:
df = spark.read.options(delimiter=',').csv(r"{0}\\{1}".format(os.getcwd(), '2005.csv'), header=False, inferSchema=True)
df.count(), len(df.columns)

(36482705, 8)

In [9]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: integer (nullable = true)



In [10]:
df.show()

+-----------+--------+----+---+----+----+---+----+
|        _c0|     _c1| _c2|_c3| _c4| _c5|_c6| _c7|
+-----------+--------+----+---+----+----+---+----+
|AE000041196|20050101|TMAX|230|null|null|  S|null|
|AE000041196|20050101|TMIN|138|null|null|  S|null|
|AE000041196|20050101|TAVG|187|   H|null|  S|null|
|AEM00041194|20050101|TMIN|146|null|null|  S|null|
|AEM00041194|20050101|PRCP|  0|null|null|  S|null|
|AEM00041194|20050101|TAVG|189|   H|null|  S|null|
|AEM00041217|20050101|TMAX|223|null|null|  S|null|
|AEM00041217|20050101|TMIN|136|null|null|  S|null|
|AEM00041217|20050101|TAVG|180|   H|null|  S|null|
|AEM00041218|20050101|TMAX|231|null|null|  S|null|
|AEM00041218|20050101|TMIN|129|null|null|  S|null|
|AEM00041218|20050101|TAVG|168|   H|null|  S|null|
|AFM00040948|20050101|TAVG| 12|   H|null|  S|null|
|AFM00040990|20050101|TAVG| 49|   H|null|  S|null|
|AG000060390|20050101|TMAX|164|null|null|  S|null|
|AG000060390|20050101|TMIN| 19|null|null|  S|null|
|AG000060390|20050101|PRCP| 30|

In [11]:
df.groupBy("_c2").count().sort(desc('count')).show()

+----+-------+
| _c2|  count|
+----+-------+
|PRCP|9159209|
|TMAX|6011301|
|TMIN|5961154|
|SNWD|4120558|
|SNOW|3678260|
|TOBS|2042269|
|TAVG|2022925|
|WESD| 348300|
|AWND| 328963|
|WDF2| 327973|
|WSF2| 327972|
|FMTM| 327329|
|WSF5| 320953|
|WDF5| 320271|
|PGTM| 201087|
|WT01| 128173|
|WT13|  71551|
|WT03|  61907|
|WT16|  61701|
|EVAP|  57774|
+----+-------+
only showing top 20 rows



In [12]:
df.count()

36482705

In [13]:
df = df.filter(col("_c2").isin(["PRCP", "TMAX", "TMIN", "SNWD", "SNOW"]))
print(df.count(), len(df.columns))

28930482 8


In [14]:
df= df.filter(col("_c0").startswith("US"))
print(df.count(), len(df.columns))

15835987 8


In [15]:
df.show()

+-----------+--------+----+---+----+----+---+----+
|        _c0|     _c1| _c2|_c3| _c4| _c5|_c6| _c7|
+-----------+--------+----+---+----+----+---+----+
|US10adam002|20050101|PRCP|  0|null|null|  N| 700|
|US10adam006|20050101|PRCP|  0|null|null|  N| 700|
|US10adam006|20050101|SNOW|  0|null|null|  N| 700|
|US10adam006|20050101|SNWD|  0|null|null|  N| 700|
|US10adam008|20050101|PRCP|  0|null|null|  N| 700|
|US10adam012|20050101|PRCP|  0|null|null|  N| 700|
|US10adam013|20050101|PRCP|  0|   T|null|  N| 700|
|US10adam013|20050101|SNOW|  0|null|null|  N| 700|
|US10adam013|20050101|SNWD|  0|null|null|  N| 700|
|US10adam016|20050101|PRCP|  0|null|null|  N| 700|
|US10adam017|20050101|PRCP|  0|null|null|  N| 700|
|US10adam017|20050101|SNOW|  0|null|null|  N| 700|
|US10adam017|20050101|SNWD|  0|null|null|  N| 700|
|US10blai005|20050101|PRCP|  0|null|null|  N| 700|
|US10blai006|20050101|PRCP|  0|null|null|  N| 700|
|US10boon002|20050101|PRCP|  0|null|null|  N| 700|
|US10boon003|20050101|PRCP|  0|

In [17]:
df= df.groupBy("_c0", "_c1").pivot("_c2").agg(first("_c3"))
df.show()

+-----------+--------+----+----+----+----+----+
|        _c0|     _c1|PRCP|SNOW|SNWD|TMAX|TMIN|
+-----------+--------+----+----+----+----+----+
|US1CODG0003|20050101|   0|   0|   0|null|null|
|USC00413873|20050101|   3|   0|   0| 244| 189|
|USC00029114|20050102|   0|   0|   0|  94|   0|
|USC00235987|20050102|   8|   0|   0| 194| 133|
|USC00218421|20050105|   0|   0| 457|-128|-289|
|USC00252065|20050106|   0|   0|  76|-150|-239|
|USC00254440|20050102|   0|   0|   0| 117| -94|
|USC00203096|20050103|   0|   0| 203|   0| -39|
|USC00204399|20050104|   0|   0|   0| -17| -61|
|USC00143954|20050105| 607|null|   0|  17|  -6|
|USC00500310|20050103|   0|   0|null| -28| -83|
|US1COEP0055|20050105|   8|  15|  25|null|null|
|USC00368888|20050105|  58|  48|  51|  83|  -6|
|USR0000CBLD|20050106|null|null|null|  -6| -56|
|USW00014742|20050106|  38|  66|  76|   0|-106|
|USC00172320|20050107|  89| 102| 102| -28|-139|
|USC00415852|20050107|   0|   0|   0|  61| -61|
|USC00238805|20050108|  28|  38|  25|  1

In [18]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- PRCP: integer (nullable = true)
 |-- SNOW: integer (nullable = true)
 |-- SNWD: integer (nullable = true)
 |-- TMAX: integer (nullable = true)
 |-- TMIN: integer (nullable = true)



In [19]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

for f in tqdm(os.listdir()):
    if f.endswith('.csv') and f!='2005.csv':
        print(f)
        d1=  spark.read.options(delimiter=',').csv(r"{0}\\{1}".format(os.getcwd(), f), header=False, inferSchema=True)
        print(d1.printSchema())
        d1 = d1.filter(col("_c2").isin(["PRCP", "TMAX", "TMIN", "SNWD", "SNOW"]))
        print(d1.count(), len(d1.columns))
        d1= d1.filter(col("_c0").startswith("US"))
        print(d1.count(), len(d1.columns))
        d1= d1.groupBy("_c0", "_c1").pivot("_c2").agg(first("_c3"))
        print(d1.count(), len(d1.columns))
        print(d1.printSchema())
        df= unionAll(df, d1)
        print(f, 'done')
        print(df.count(), len(df.columns))

  0%|          | 0/19 [00:00<?, ?it/s]

2006.csv
root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: integer (nullable = true)

None
29246562 8
16387595 8
4540893 7
root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- PRCP: integer (nullable = true)
 |-- SNOW: integer (nullable = true)
 |-- SNWD: integer (nullable = true)
 |-- TMAX: integer (nullable = true)
 |-- TMIN: integer (nullable = true)

None
2006.csv done


 11%|█         | 2/19 [00:53<07:35, 26.80s/it]

8833033 7
2007.csv
root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: integer (nullable = true)

None
29663095 8
17361672 8
5045671 7
root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- PRCP: integer (nullable = true)
 |-- SNOW: integer (nullable = true)
 |-- SNWD: integer (nullable = true)
 |-- TMAX: integer (nullable = true)
 |-- TMIN: integer (nullable = true)

None
2007.csv done


 16%|█▌        | 3/19 [01:47<10:10, 38.17s/it]

...

116281854 7
2022.csv
root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: integer (nullable = true)

None
28536488 8
20708508 8
8284369 7
root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- PRCP: integer (nullable = true)
 |-- SNOW: integer (nullable = true)
 |-- SNWD: integer (nullable = true)
 |-- TMAX: integer (nullable = true)
 |-- TMIN: integer (nullable = true)

None
2022.csv done


 95%|█████████▍| 18/19 [26:10<01:59, 119.53s/it]

124566223 7
2023.csv
root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: integer (nullable = true)

None
8227703 8
6052157 8
2431258 7
root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- PRCP: integer (nullable = true)
 |-- SNOW: integer (nullable = true)
 |-- SNWD: integer (nullable = true)
 |-- TMAX: integer (nullable = true)
 |-- TMIN: integer (nullable = true)

None
2023.csv done


100%|██████████| 19/19 [28:24<00:00, 89.72s/it] 

126997481 7


In [20]:
for col in df.columns:
    null_count = df.where(df[col].isNull()).count()
    print(col, null_count)

_c0 0
_c1 0
PRCP 9663397
SNOW 50603599
SNWD 80674764
TMAX 74245243
TMIN 74285055


In [21]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- PRCP: integer (nullable = true)
 |-- SNOW: integer (nullable = true)
 |-- SNWD: integer (nullable = true)
 |-- TMAX: integer (nullable = true)
 |-- TMIN: integer (nullable = true)



In [22]:
print(df.count(), len(df.columns))

126997481 7


In [24]:
dref= spark.read.options(delimiter=',').csv(r"{0}\\{1}".format(os.getcwd(), '../noaa-ghcnd-stations.txt'), header=False, inferSchema=True)
dref= dref.drop(dref._c3)
dref= dref.drop(dref._c5)
dref= dref.drop(dref._c6)
dref= dref.drop(dref._c7)
dref= dref.withColumnRenamed('_c1', 'lat')
dref= dref.withColumnRenamed('_c2', 'lng')
dref= dref.withColumnRenamed('_c4', 'state')
dref.show()

+-----------+-------+---------+-----+
|        _c0|    lat|      lng|state|
+-----------+-------+---------+-----+
|US10RMHS145|40.5268|-105.1113|   CO|
|US10adam001| 40.568| -98.5069|   NE|
|US10adam002|40.5093| -98.5493|   NE|
|US10adam003|40.4663| -98.6537|   NE|
|US10adam004|40.4798| -98.4026|   NE|
|US10adam006|40.4372| -98.5912|   NE|
|US10adam007|40.5389| -98.4713|   NE|
|US10adam008|40.4953| -98.2973|   NE|
|US10adam010|40.5532| -98.6297|   NE|
|US10adam011|40.4078| -98.6161|   NE|
|US10adam012|  40.62|   -98.39|   NE|
|US10adam013|  40.66|   -98.42|   NE|
|US10adam015|40.6258| -98.7034|   NE|
|US10adam016|  40.64| -98.3949|   NE|
|US10adam017|40.6567| -98.4789|   NE|
|US10adam019|40.6114| -98.5543|   NE|
|US10adam022|40.5936| -98.4299|   NE|
|US10adam023|40.5981| -98.4732|   NE|
|US10adam024|40.3901| -98.2715|   NE|
|US10adam025|40.6004| -98.3986|   NE|
+-----------+-------+---------+-----+
only showing top 20 rows



In [25]:
df.show()

+-----------+--------+----+----+----+----+----+
|        _c0|     _c1|PRCP|SNOW|SNWD|TMAX|TMIN|
+-----------+--------+----+----+----+----+----+
|US1CODG0003|20050101|   0|   0|   0|null|null|
|USC00413873|20050101|   3|   0|   0| 244| 189|
|USC00029114|20050102|   0|   0|   0|  94|   0|
|USC00235987|20050102|   8|   0|   0| 194| 133|
|USC00218421|20050105|   0|   0| 457|-128|-289|
|USC00252065|20050106|   0|   0|  76|-150|-239|
|USC00254440|20050102|   0|   0|   0| 117| -94|
|USC00203096|20050103|   0|   0| 203|   0| -39|
|USC00204399|20050104|   0|   0|   0| -17| -61|
|USC00143954|20050105| 607|null|   0|  17|  -6|
|USC00500310|20050103|   0|   0|null| -28| -83|
|US1COEP0055|20050105|   8|  15|  25|null|null|
|USC00368888|20050105|  58|  48|  51|  83|  -6|
|USR0000CBLD|20050106|null|null|null|  -6| -56|
|USW00014742|20050106|  38|  66|  76|   0|-106|
|USC00172320|20050107|  89| 102| 102| -28|-139|
|USC00415852|20050107|   0|   0|   0|  61| -61|
|USC00238805|20050108|  28|  38|  25|  1

In [26]:
del d1

In [27]:
print(dref.count(), len(dref.columns))
print(df.count(), len(df.columns))

70627 4
126997481 7


In [28]:
# Define the common column(s) to join the dataframes on
join_columns = ["_c0"]

# Merge the dataframes using the join function
merged_df = df.join(dref, join_columns, "left")
print(merged_df.count(), len(merged_df.columns))

126997481 10


In [29]:
merged_df.show()

+-----------+--------+----+----+----+----+----+-------+---------+-----+
|        _c0|     _c1|PRCP|SNOW|SNWD|TMAX|TMIN|    lat|      lng|state|
+-----------+--------+----+----+----+----+----+-------+---------+-----+
|US1CODG0003|20050101|   0|   0|   0|null|null|39.4794|-105.0055|   CO|
|USC00413873|20050101|   3|   0|   0| 244| 189|29.4706| -96.9397|   TX|
|USC00029114|20050102|   0|   0|   0|  94|   0|36.9953|-111.4914|   AZ|
|USC00235987|20050102|   8|   0|   0| 194| 133|37.8394| -94.3739|   MO|
|USC00218421|20050105|   0|   0| 457|-128|-289|47.1253| -91.7067|   MN|
|USC00252065|20050106|   0|   0|  76|-150|-239|40.2317|-100.8303|   NE|
|USC00254440|20050102|   0|   0|   0| 117| -94|41.2453|-103.6344|   NE|
|USC00203096|20050103|   0|   0| 203|   0| -39|45.0333| -84.7114|   MI|
|USC00204399|20050104|   0|   0|   0| -17| -61|44.5694| -85.5722|   MI|
|USC00143954|20050105| 607|null|   0|  17|  -6|37.2364| -95.7003|   KS|
|USC00500310|20050103|   0|   0|null| -28| -83|57.4989|-134.5864

In [30]:
merged_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- PRCP: integer (nullable = true)
 |-- SNOW: integer (nullable = true)
 |-- SNWD: integer (nullable = true)
 |-- TMAX: integer (nullable = true)
 |-- TMIN: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- state: string (nullable = true)



In [31]:
merged_df.groupBy("state").count().sort(desc('count')).show()

+-----+-------+
|state|  count|
+-----+-------+
|   TX|9963807|
|   CO|7672148|
|   CA|6591627|
|   NE|4643570|
|   OR|4284497|
|   NC|4214640|
|   KS|4090095|
|   IL|3818867|
|   FL|3707986|
|   NM|3432126|
|   MT|3342106|
|   MO|3269779|
|   WA|3192283|
|   AZ|3148082|
|   IN|2976566|
|   TN|2892980|
|   MN|2718420|
|   NY|2615803|
|   WY|2586399|
|   WI|2456009|
+-----+-------+
only showing top 20 rows



In [32]:
cadf= merged_df.filter("state == 'CA'")
cadf.count()

6591627

In [1]:
del merged_df, df

In [33]:
chk= pd.read_csv('2005.csv')
chk

,AE000041196,20050101,TMAX,230,Unnamed: 4,Unnamed: 5,S,Unnamed: 7
0,AE000041196,20050101,TMIN,138,NaN,NaN,S,NaN
1,AE000041196,20050101,TAVG,187,H,NaN,S,NaN
2,AEM00041194,20050101,TMIN,146,NaN,NaN,S,NaN
3,AEM00041194,20050101,PRCP,0,NaN,NaN,S,NaN
4,AEM00041194,20050101,TAVG,189,H,NaN,S,NaN
...,...,...,...,...,...,...,...,...
36482699,ZI000067975,20051231,TAVG,211,H,NaN,S,NaN
36482700,ZI000067983,20051231,TMAX,232,NaN,NaN,S,NaN
36482701,ZI000067983,20051231,TMIN,154,NaN,NaN,S,NaN
36482702,ZI000067983,20051231,PRCP,191,NaN,NaN,S,NaN


In [34]:
chk[(chk['AE000041196']=='USS0019L24S') & (chk['20050101']==20050128)]

,AE000041196,20050101,TMAX,230,Unnamed: 4,Unnamed: 5,S,Unnamed: 7
2781676,USS0019L24S,20050128,TMAX,-142,NaN,I,T,NaN
2781677,USS0019L24S,20050128,TMIN,-149,NaN,NaN,T,NaN
2781678,USS0019L24S,20050128,TOBS,-147,NaN,NaN,T,NaN
2781679,USS0019L24S,20050128,PRCP,0,NaN,NaN,T,NaN
2781680,USS0019L24S,20050128,SNWD,1981,NaN,NaN,T,NaN
2781681,USS0019L24S,20050128,TAVG,-146,NaN,NaN,T,NaN
2781682,USS0019L24S,20050128,WESD,6172,NaN,NaN,T,NaN


In [35]:
cadf.show()

+-----------+--------+----+----+----+----+----+-------+---------+-----+
|        _c0|     _c1|PRCP|SNOW|SNWD|TMAX|TMIN|    lat|      lng|state|
+-----------+--------+----+----+----+----+----+-------+---------+-----+
|USR0000CBLD|20050106|null|null|null|  -6| -56|41.0547|-120.3375|   CA|
|USR0000CSRI|20050106|null|null|null| 122|  72|33.9778|-120.0778|   CA|
|USW00003178|20050111| 236|null|null| 156|  94|32.5758|-116.9939|   CA|
|USR0000CSLA|20050108|null|null|null| -33| -50|41.8586|-123.3525|   CA|
|USC00041277|20050123|   0|   0|null| 106|  11|38.2769|-120.3114|   CA|
|USC00046136|20050124|   0|   0|   0| 183|  56|39.2467|-121.0008|   CA|
|USS0019L24S|20050128|   0|null|1981|-142|-149|  38.92|  -119.92|   CA|
|USC00045449|20050131|   0|   0| 356| 128| -44|41.2517|-122.1383|   CA|
|USR0000CCOU|20050204|null|null|null| 161|  50|39.0189|-122.4119|   CA|
|USC00040161|20050206|   0|   0|   0|  39|-100|41.4903|-120.5439|   CA|
|USR0000CJOH|20050220|null|null|null| 111|   0|35.9717| -118.545

In [36]:
for col in cadf.columns:
    null_count = cadf.where(cadf[col].isNull()).count()
    print(col, null_count)

_c0 0
_c1 0
PRCP 1831646
SNOW 3801349
SNWD 5047928
TMAX 2582044
TMIN 2585687
lat 0
lng 0
state 0


In [37]:
cadf.groupBy("_c0").count().sort(desc('count')).show()

+-----------+-----+
|        _c0|count|
+-----------+-----+
|USS0019L13S| 6690|
|USW00023273| 6690|
|USS0019L19S| 6690|
|USW00023213| 6690|
|USS0019L17S| 6690|
|USS0019L24S| 6690|
|USW00023237| 6690|
|USW00023285| 6690|
|USS0020K04S| 6690|
|USW00023234| 6690|
|USS0019L05S| 6690|
|USS0020K31S| 6690|
|USW00023174| 6690|
|USW00003159| 6690|
|USW00023155| 6690|
|USS0019L42S| 6690|
|USW00023258| 6690|
|USS0020K25S| 6690|
|USW00023190| 6690|
|USS0019L43S| 6690|
+-----------+-----+
only showing top 20 rows



In [38]:
365*19

6935

In [39]:
6935*.75

5201.25

In [40]:
cadf.groupBy("_c0").count().sort('count').show(200)

+-----------+-----+
|        _c0|count|
+-----------+-----+
|USC00042084|    1|
|USC00041886|    1|
|USC00044883|    1|
|USC00043194|    1|
|USR0000CBLR|    1|
|USC00045711|    1|
|USC00045242|    1|
|USC00044074|    1|
|USC00042749|    1|
|USW00023202|    1|
|USC00043891|    1|
|USC00042350|    3|
|USC00048025|   33|
|USC00040235|   59|
|USC00048406|   61|
|USC00044705|   61|
|USR0000CFSL|   75|
|USC00047123|   90|
|USC00046473|   90|
|USC00043285|   90|
|USC00041057|   90|
|USC00048243|   90|
|USC00041518|   90|
|USC00042756|   92|
|US1CASZ0043|  100|
|US1CASB0020|  100|
|US1CAYB0002|  102|
|US1CASZ0040|  103|
|US1CASN0052|  103|
|US1CASZ0025|  104|
|US1CAHM0027|  105|
|US1CAMA0006|  105|
|US1CATY0025|  106|
|US1CASD0202|  106|
|US1CASN0108|  107|
|US1CAHM0116|  107|
|US1CALA0013|  108|
|US1CASL0011|  111|
|US1CASH0014|  111|
|US1CASN0041|  111|
|US1CAMT0001|  112|
|US1CAVT0019|  113|
|US1CASH0035|  114|
|US1CATM0011|  114|
|US1CAYL0025|  114|
|US1CAHM0064|  114|
|US1CAIN0011|  115|


In [41]:
print(cadf.count(), len(cadf.columns))

6591627 10


In [42]:
drp= cadf.toPandas()
drp.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6591627 entries, 0 to 6591626
Data columns (total 10 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   _c0     6591627 non-null  object 
 1   _c1     6591627 non-null  int32  
 2   PRCP    4759981 non-null  float64
 3   SNOW    2790278 non-null  float64
 4   SNWD    1543699 non-null  float64
 5   TMAX    4009583 non-null  float64
 6   TMIN    4005940 non-null  float64
 7   lat     6591627 non-null  float64
 8   lng     6591627 non-null  float64
 9   state   6591627 non-null  object 
dtypes: float64(7), int32(1), object(2)
memory usage: 477.8+ MB


In [43]:
drp.head()

,_c0,_c1,PRCP,SNOW,SNWD,TMAX,TMIN,lat,lng,state
0,USR0000CBLD,20050106,NaN,NaN,NaN,-6.0,-56.0,41.0547,-120.3375,CA
1,USR0000CSRI,20050106,NaN,NaN,NaN,122.0,72.0,33.9778,-120.0778,CA
2,USW00003178,20050111,236.0,NaN,NaN,156.0,94.0,32.5758,-116.9939,CA
3,USR0000CSLA,20050108,NaN,NaN,NaN,-33.0,-50.0,41.8586,-123.3525,CA
4,USC00041277,20050123,0.0,0.0,NaN,106.0,11.0,38.2769,-120.3114,CA


In [44]:
c= drp['_c0'].value_counts()
c= pd.DataFrame(c)
c

,_c0
USS0020L02S,6690
USW00093209,6690
USS0019L08S,6690
USS0020K04S,6690
USS0019L38S,6690
...,...
USC00042084,1
USC00044074,1
USC00043891,1
USC00044883,1


In [45]:
c['_c0'].value_counts().sort_index()

1       11
3        1
33       1
59       1
61       2
        ..
6686     1
6687     5
6688     9
6689    11
6690    55
Name: _c0, Length: 1593, dtype: int64

In [47]:
drp.to_csv('../../AWS/glueInput/noaa_ca_2005_2023_station_day_level.csv', index=False)

In [48]:
drp.sort_values(by=['_c0', '_c1'], ascending=[True, True], inplace=True)

In [49]:
pd.to_datetime(drp['_c1'], format='%Y%m%d').dt.year.value_counts()

2016    412421
2017    405708
2015    404005
2014    400127
2018    399495
2019    398506
2020    398425
2022    386550
2013    383322
2012    373991
2010    372062
2011    364916
2009    353959
2021    332171
2008    275097
2005    272304
2007    268619
2006    268503
2023    121446
Name: _c1, dtype: int64

In [50]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

In [51]:
dff= drp.copy()

In [52]:
from geopy.geocoders import Nominatim

# create a geocoder object
geolocator = Nominatim(user_agent='myapp')

# define a function to get the county name and FIPS county code from latitude and longitude
def get_county(lat, lon):
    location = geolocator.reverse(f'{lat}, {lon}')
    address = location.raw['address']
    county_name = address.get('county')
    fips_code = address.get('county_code')
    return county_name, fips_code

In [53]:
s= drp[['_c0', 'lat', 'lng']]
s.drop_duplicates(inplace=True)
s

C:\Users\31405.ISBDOMAIN1\AppData\Local\Temp\12\ipykernel_21584\307963564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s.drop_duplicates(inplace=True)


,_c0,lat,lng
1039837,US1CAAL0001,37.8123,-122.2160
909217,US1CAAL0002,37.7075,-122.0687
1268865,US1CAAL0003,37.7169,-122.0585
1075625,US1CAAL0004,37.6483,-121.8745
1715418,US1CAAL0006,37.8319,-122.2503
...,...,...,...
91606,USW00093241,38.3775,-121.9586
10440,USW00093242,36.9850,-120.1106
16127,USW00093243,37.2381,-120.8825
831781,USW00093245,38.3208,-123.0747


In [56]:
s

,_c0,lat,lng
1039837,US1CAAL0001,37.8123,-122.2160
909217,US1CAAL0002,37.7075,-122.0687
1268865,US1CAAL0003,37.7169,-122.0585
1075625,US1CAAL0004,37.6483,-121.8745
1715418,US1CAAL0006,37.8319,-122.2503
...,...,...,...
91606,USW00093241,38.3775,-121.9586
10440,USW00093242,36.9850,-120.1106
16127,USW00093243,37.2381,-120.8825
831781,USW00093245,38.3208,-123.0747


In [57]:
s[['county_name', 'fips_code']] = s.apply(lambda x: get_county(x['lat'], x['lng']), axis=1, result_type='expand')

C:\Users\31405.ISBDOMAIN1\AppData\Local\Temp\12\ipykernel_21584\2753543191.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[['county_name', 'fips_code']] = s.apply(lambda x: get_county(x['lat'], x['lng']), axis=1, result_type='expand')
C:\Users\31405.ISBDOMAIN1\AppData\Local\Temp\12\ipykernel_21584\2753543191.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[['county_name', 'fips_code']] = s.apply(lambda x: get_county(x['lat'], x['lng']), axis=1, result_type='expand')


In [58]:
s

,_c0,lat,lng,county_name,fips_code
1039837,US1CAAL0001,37.8123,-122.2160,Alameda County,None
909217,US1CAAL0002,37.7075,-122.0687,Alameda County,None
1268865,US1CAAL0003,37.7169,-122.0585,Alameda County,None
1075625,US1CAAL0004,37.6483,-121.8745,Alameda County,None
1715418,US1CAAL0006,37.8319,-122.2503,Alameda County,None
...,...,...,...,...,...
91606,USW00093241,38.3775,-121.9586,Solano County,None
10440,USW00093242,36.9850,-120.1106,Madera County,None
16127,USW00093243,37.2381,-120.8825,Merced County,None
831781,USW00093245,38.3208,-123.0747,Sonoma County,None


In [59]:
s.drop_duplicates(subset=['_c0'])

,_c0,lat,lng,county_name,fips_code
1039837,US1CAAL0001,37.8123,-122.2160,Alameda County,None
909217,US1CAAL0002,37.7075,-122.0687,Alameda County,None
1268865,US1CAAL0003,37.7169,-122.0585,Alameda County,None
1075625,US1CAAL0004,37.6483,-121.8745,Alameda County,None
1715418,US1CAAL0006,37.8319,-122.2503,Alameda County,None
...,...,...,...,...,...
91606,USW00093241,38.3775,-121.9586,Solano County,None
10440,USW00093242,36.9850,-120.1106,Madera County,None
16127,USW00093243,37.2381,-120.8825,Merced County,None
831781,USW00093245,38.3208,-123.0747,Sonoma County,None


In [60]:
s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2193 entries, 1039837 to 62427
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _c0          2193 non-null   object 
 1   lat          2193 non-null   float64
 2   lng          2193 non-null   float64
 3   county_name  2163 non-null   object 
 4   fips_code    0 non-null      object 
dtypes: float64(2), object(3)
memory usage: 102.8+ KB


In [62]:
s.drop(columns=['fips_code'], inplace=True)

C:\Users\31405.ISBDOMAIN1\AppData\Local\Temp\12\ipykernel_21584\743171589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s.drop(columns=['fips_code'], inplace=True)


In [63]:
cf= pd.read_excel('../county_fips.xlsx')
cf.columns=['county_name', 'fips_code']
cf['county_name']= cf['county_name'].apply(lambda x: x+' County')
cf

,county_name,fips_code
0,Alameda County,1
1,Alpine County,3
2,Amador County,5
3,Butte County,7
4,Calaveras County,9
5,Colusa County,11
6,Contra Costa County,13
7,Del Norte County,15
8,El Dorado County,17
9,Fresno County,19


In [65]:
s

,_c0,lat,lng,county_name
1039837,US1CAAL0001,37.8123,-122.2160,Alameda County
909217,US1CAAL0002,37.7075,-122.0687,Alameda County
1268865,US1CAAL0003,37.7169,-122.0585,Alameda County
1075625,US1CAAL0004,37.6483,-121.8745,Alameda County
1715418,US1CAAL0006,37.8319,-122.2503,Alameda County
...,...,...,...,...
91606,USW00093241,38.3775,-121.9586,Solano County
10440,USW00093242,36.9850,-120.1106,Madera County
16127,USW00093243,37.2381,-120.8825,Merced County
831781,USW00093245,38.3208,-123.0747,Sonoma County


In [68]:
s= pd.merge(s, cf, how='left')
s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2193 entries, 0 to 2192
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _c0          2193 non-null   object 
 1   lat          2193 non-null   float64
 2   lng          2193 non-null   float64
 3   county_name  2163 non-null   object 
 4   fips_code    2058 non-null   float64
dtypes: float64(3), object(2)
memory usage: 102.8+ KB


In [69]:
s[s['fips_code'].isna()]

,_c0,lat,lng,county_name,fips_code
242,US1CAIN0001,37.3626,-118.3883,CAL Fire San Bernardino Unit,NaN
244,US1CAIN0004,37.3878,-118.4141,CAL Fire San Bernardino Unit,NaN
247,US1CAIN0007,37.3854,-118.4150,CAL Fire San Bernardino Unit,NaN
248,US1CAIN0008,36.8061,-118.2014,CAL Fire San Bernardino Unit,NaN
249,US1CAIN0009,37.1688,-118.2878,CAL Fire San Bernardino Unit,NaN
...,...,...,...,...,...
2122,USW00023179,34.7678,-114.6183,CAL Fire San Bernardino Unit,NaN
2144,USW00023272,37.7706,-122.4269,None,NaN
2160,USW00053139,36.6019,-117.1450,None,NaN
2166,USW00093104,35.6864,-117.6908,None,NaN


In [70]:
drp

,_c0,_c1,PRCP,SNOW,SNWD,TMAX,TMIN,lat,lng,state
1039837,US1CAAL0001,20081007,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA
1062617,US1CAAL0001,20081008,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA
812121,US1CAAL0001,20081009,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA
1082561,US1CAAL0001,20081010,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA
981546,US1CAAL0001,20081011,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA
...,...,...,...,...,...,...,...,...,...,...
6479302,USW00094299,20230422,0.0,NaN,NaN,200.0,-27.0,41.4836,-120.5614,CA
6471043,USW00094299,20230423,0.0,NaN,NaN,194.0,50.0,41.4836,-120.5614,CA
6496148,USW00094299,20230424,0.0,NaN,NaN,167.0,-16.0,41.4836,-120.5614,CA
6532286,USW00094299,20230425,0.0,NaN,NaN,200.0,-38.0,41.4836,-120.5614,CA


In [71]:
pd.merge(drp, s)

,_c0,_c1,PRCP,SNOW,SNWD,TMAX,TMIN,lat,lng,state,county_name,fips_code
0,US1CAAL0001,20081007,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
1,US1CAAL0001,20081008,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
2,US1CAAL0001,20081009,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
3,US1CAAL0001,20081010,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
4,US1CAAL0001,20081011,0.0,0.0,NaN,NaN,NaN,37.8123,-122.2160,CA,Alameda County,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6591622,USW00094299,20230422,0.0,NaN,NaN,200.0,-27.0,41.4836,-120.5614,CA,Modoc County,49.0
6591623,USW00094299,20230423,0.0,NaN,NaN,194.0,50.0,41.4836,-120.5614,CA,Modoc County,49.0
6591624,USW00094299,20230424,0.0,NaN,NaN,167.0,-16.0,41.4836,-120.5614,CA,Modoc County,49.0
6591625,USW00094299,20230425,0.0,NaN,NaN,200.0,-38.0,41.4836,-120.5614,CA,Modoc County,49.0


In [72]:
dff= pd.merge(drp, s)
dff.to_csv('../noaa_ca_2005_2023_station_day_level.csv', index=False)